In [2]:
import os
import json
import psycopg2
import pandas as pd
import geopandas as gpd
from geopandas import GeoSeries, GeoDataFrame
import folium
import fiona
from pyproj import Proj, transform
import osmnx as ox

In [3]:
conn = psycopg2.connect(dbname="gis", user="postgres", password="")
ox.config(log_file=True, log_console=True, use_cache=True)

In [20]:
rapperswil_polygon_query = "SELECT way AS geometry FROM planet_osm_polygon WHERE osm_id = -1683921"

polygon_gdf = gpd.read_postgis(rapperswil_polygon_query, conn, geom_col='geometry')
polygon_gdf.crs = fiona.crs.from_epsg('3857')
polygon_gdf = polygon_gdf.to_crs(fiona.crs.from_epsg('4326'))
polygon = polygon_gdf['geometry'][0]

In [21]:
from osmnx import projection

projection.project_geometry(polygon, crs=polygon_gdf.crs)

(<shapely.geometry.polygon.Polygon at 0x7f4061024940>,
 {'datum': 'WGS84', 'ellps': 'WGS84', 'proj': 'utm', 'units': 'm', 'zone': 32})

In [ ]:
rapperswil_graph = ox.graph_from_polygon(polygon, network_type='all')
rapperswil_graph_projected = ox.project_graph(rapperswil_graph)
fig, ax = ox.plot_graph(rapperswil_graph_projected)

In [ ]:
ox.plot_graph_folium(rapperswil_graph, popup_attribute='name', edge_width=2)

In [26]:
# define a bounding box of Rapperswil Altstadt
north, south, east, west = 8.8120980802, 47.2238442691, 8.8231165948, 47.2284417489

# create network from that bounding box
G1 = ox.graph_from_bbox(north, south, east, west, network_type='drive_service')

KeyboardInterrupt: 